In [ ]:
#%env GOOGLE_APPLICATION_CREDENTIALS=/home/pitfox/data/spark-container-dev-f5d53ab2439c.json

In [ ]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
!echo $SPARK_HOME

In [ ]:
!cat $SPARK_HOME/conf/spark-defaults.conf

In [7]:
# import findspark
# findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, BooleanType, ArrayType, LongType
import pyspark.sql.functions as f
import re
import requests
from bs4 import BeautifulSoup

In [ ]:
#spark.stop()

In [8]:
spark = SparkSession \
    .builder \
    .appName("Load Review Files") \
    .config("spark.jars", "/opt/spark/jars/spark-bigquery-with-dependencies_2.12-0.20.0.jar") \
    .getOrCreate()

In [9]:
# Update to your GCS bucket
gcs_bucket = f'amazon_reviews_bucket'
gcs_filepath = f'gs://amazon_reviews_bucket'

In [10]:
# Define schema of files to parse
schema = StructType([ 
    StructField("asin",StringType(),True), 
    StructField("image",ArrayType(StringType()),True), 
    StructField("overall",DoubleType(),True),
    StructField("reviewText",StringType(),True),
    StructField("reviewTime",StringType(),True),
    StructField("reviewerID",StringType(),True),
    StructField("reviewerName",StringType(),True),
    StructField("summary",StringType(),True),
    StructField("unixReviewTime",LongType(),True),
    StructField("verified",BooleanType(),True),
    StructField("vote",StringType(),True),
    StructField("style",ArrayType(StringType()),True)
  ])

In [29]:
# URL to scrape to get files to download
url = "https://nijianmo.github.io/amazon/index.html"
html = requests.get(url)

if html.ok:
    soup = BeautifulSoup(html.content, 'html.parser')  

output_final = []
files = []
links = soup.find_all('a',string='5-core')#.find('5-core')#.find_all('td', id='5-core')
for link in links:
    url = link.get('href')
    file = url.split('/')[-1]
    print(url)
    print(url.split('/')[-1])
    spark.sparkContext.addFile(url)
    files.append(file)
    df = spark.read.json("file://"+SparkFiles.get(file),schema)

http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/AMAZON_FASHION_5.json.gz
AMAZON_FASHION_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty_5.json.gz
All_Beauty_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Appliances_5.json.gz
Appliances_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Arts_Crafts_and_Sewing_5.json.gz
Arts_Crafts_and_Sewing_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Automotive_5.json.gz
Automotive_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Books_5.json.gz
Books_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/CDs_and_Vinyl_5.json.gz
CDs_and_Vinyl_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Cell_Phones_and_Accessories_5.json.gz
Cell_Phones_and_Accessories_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Clothing_Shoes_and_Jewelry_5.json.gz
Clothing_Shoes_and_Jewelry_5.json.gz
http:/

In [5]:
# URL to scrape to get files to download
url = "https://nijianmo.github.io/amazon/index.html"
html = requests.get(url)

if html.ok:
    soup = BeautifulSoup(html.content, 'html.parser')  

output_final = []
files = []
links = soup.find_all('a',string='5-core')#.find('5-core')#.find_all('td', id='5-core')
for link in links:
    url = link.get('href')
    file = url.split('/')[-1]
    print(url)
    print(url.split('/')[-1])
    spark.sparkContext.addFile(url)
    files.append(file)

for file in files:    
    df = spark.read.json("file://"+SparkFiles.get(file),schema)
    df = df.dropDuplicates() 
    df = df.withColumn('review_wordCount', f.size(f.split(f.col('reviewText'), ' ')))
    df.registerTempTable("dataframe")
    sql_script = f"""select 
              '{file}' as category,
              asin || '-' || reviewerID || row_number() OVER (PARTITION BY asin, reviewerID ORDER BY unixReviewTime asc) as review_ID,
              asin as product_ID,
              reviewerID as reviewer_ID,
              overall as rating_out_of_5,
              summary as review_summary,
              reviewText as review_text,
              review_wordCount as review_word_count,     
              '{url}' as source_url
            from dataframe"""
    output = spark.sql(sql_script)
    if not output_final:
        output_final = output
        print(f'loaded first file: {file}')
    else:
        output_final = output_final.union(output)
        print(f'appended to df with: {file}')  

http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/AMAZON_FASHION_5.json.gz
AMAZON_FASHION_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty_5.json.gz
All_Beauty_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Appliances_5.json.gz
Appliances_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Arts_Crafts_and_Sewing_5.json.gz
Arts_Crafts_and_Sewing_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Automotive_5.json.gz
Automotive_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Books_5.json.gz
Books_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/CDs_and_Vinyl_5.json.gz
CDs_and_Vinyl_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Cell_Phones_and_Accessories_5.json.gz
Cell_Phones_and_Accessories_5.json.gz
http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Clothing_Shoes_and_Jewelry_5.json.gz
Clothing_Shoes_and_Jewelry_5.json.gz
http:/

In [ ]:
output_final.show(5)

In [6]:
output_final.write \
  .format("bigquery") \
  .option("temporaryGcsBucket",gcs_bucket) \
  .mode("append") \
  .save("amazon_reviews.categoryFilesSmall")

Py4JJavaError: An error occurred while calling o558.save.
: java.lang.RuntimeException: Failed to write to BigQuery
	at com.google.cloud.spark.bigquery.BigQueryWriteHelper.writeDataFrameToBigQuery(BigQueryWriteHelper.scala:94)
	at com.google.cloud.spark.bigquery.BigQueryInsertableRelation.insert(BigQueryInsertableRelation.scala:43)
	at com.google.cloud.spark.bigquery.BigQueryRelationProvider.createRelation(BigQueryRelationProvider.scala:112)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:46)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:90)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$e

In [ ]:
# spark.sparkContext._jvm.scala.util.Properties.versionString()

In [ ]:
category = ['AMAZON_FASHION','All_Beauty']

In [ ]:
url = f"http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/{category[1]}_5.json.gz"

In [ ]:
spark.sparkContext.addFile(url)

In [ ]:
df = spark.read.json("file://"+SparkFiles.get(f"{category[1]}_5.json.gz"))

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, BooleanType, ArrayType, LongType

In [ ]:
schema = StructType([ 
    StructField("asin",StringType(),True), 
    StructField("image",ArrayType(StringType()),True), 
    StructField("overall",DoubleType(),True),
    StructField("reviewText",StringType(),True),
    StructField("reviewTime",StringType(),True),
    StructField("reviewerID",StringType(),True),
    StructField("reviewerName",StringType(),True),
    StructField("summary",StringType(),True),
    StructField("unixReviewedTime",LongType(),True),
    StructField("verified",BooleanType(),True),
    StructField("vote",StringType(),True)
  ])

In [ ]:
df.show(10)

In [ ]:
df = df.dropDuplicates() 

In [ ]:
df = df.dropDuplicates() 
df = df.withColumn('review_wordCount', f.size(f.split(f.col('reviewText'), ' ')))
df.registerTempTable("dataframe")
sql_script = f"""select 
          '{category[1]}' as category,
          asin || '-' || reviewerID || row_number() OVER (PARTITION BY asin, reviewerID ORDER BY unixReviewTime asc) as review_ID,
          asin as product_ID,
          reviewerID as reviewer_ID,
          overall as rating_out_of_5,
          summary as review_summary,
          reviewText as review_text,
          review_wordCount as review_word_count,     
          '{url}' as source_url
        from dataframe"""
output = spark.sql(sql_script)


In [ ]:
df.registerTempTable("dataframe")

In [ ]:
sql_script = f"""select 
          '{category[1]}' as category,
          asin || '-' || reviewerID || row_number() OVER (PARTITION BY asin, reviewerID ORDER BY unixReviewTime asc) as review_ID,
          asin as product_ID,
          reviewerID as reviewer_ID,
          overall as rating_out_of_5,
          summary as review_summary,
          reviewText as review_text,
          review_wordCount as review_word_count,     
          '{url}' as source_url
        from dataframe"""

In [ ]:
output = spark.sql(sql_script)

In [ ]:
output.show(3)

In [ ]:
# Update to your GCS bucket
gcs_bucket = f'amazon_reviews_bucket'

gcs_filepath = f'gs://amazon_reviews_bucket'

In [ ]:
output.write \
  .format("bigquery") \
  .option("temporaryGcsBucket",gcs_bucket) \
  .mode("append") \
  .save("amazon_reviews.categoryFilesSmall")

In [ ]:
# output.write.partitionBy("product_ID").csv(gcs_filepath)

In [ ]:
# output.write \
#   .mode('overwrite') \
#   .csv(gcs_filepath)

In [1]:
import wget

In [2]:
url = 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/AMAZON_FASHION_5.json.gz'
wget.download(url)

'AMAZON_FASHION_5.json.gz'

In [4]:
file = 'AMAZON_FASHION_5.json.gz'

In [5]:
import gzip
import shutil
with gzip.open(file, 'rb') as f_in:
    with open('.'.join(file.split('.')[0:2]), 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
file

In [ ]:
'.'.join(file.split('.')[0:2])

In [12]:
spark.sparkContext.addFile(url)

In [24]:
df = spark.read.json("file://"+SparkFiles.get(file),schema)

In [25]:
df.show(5)

+----------+-----+-------+--------------------+----------+-------------+------------+----------+--------------+--------+----+
|      asin|image|overall|          reviewText|reviewTime|   reviewerID|reviewerName|   summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+----------+-------------+------------+----------+--------------+--------+----+
|B000K2PJ4K| null|    5.0|Great product and...|09 4, 2015|ALJ66O1Y6SLHA|    Tonya B.|Five Stars|    1441324800|    true|null|
|B000K2PJ4K| null|    5.0|Great product and...|09 4, 2015|ALJ66O1Y6SLHA|    Tonya B.|Five Stars|    1441324800|    true|null|
|B000K2PJ4K| null|    5.0|Great product and...|09 4, 2015|ALJ66O1Y6SLHA|    Tonya B.|Five Stars|    1441324800|    true|null|
|B000K2PJ4K| null|    5.0|Great product and...|09 4, 2015|ALJ66O1Y6SLHA|    Tonya B.|Five Stars|    1441324800|    true|null|
|B000K2PJ4K| null|    5.0|Great product and...|09 4, 2015|ALJ66O1Y6SLHA|    Tonya B.|Five Stars|    1441324800|    tru

In [16]:
df.count()

3176

In [ ]:
df.dropDuplicates()

In [26]:
df = spark.read.json("file://"+SparkFiles.get(file),schema)
df = df.dropDuplicates(['reviewerID','overall','summary','reviewText']) 
df = df.withColumn('review_wordCount', f.size(f.split(f.col('reviewText'), ' ')))
df.registerTempTable("dataframe")
sql_script = f"""select 
          '{file.split('.')[0]}' as category,
          asin || '-' || reviewerID || '-' || row_number() OVER (PARTITION BY reviewerID ORDER BY unixReviewTime asc) as review_ID,
          asin as product_ID,
          reviewerID as reviewer_ID,
          overall as rating_out_of_5,
          summary as review_summary,
          reviewText as review_text,
          review_wordCount as review_word_count,  
          verified,
          vote,
          reviewTime,
          unixReviewTime,
          image,
          '{url}' as source_url
        from dataframe"""
output = spark.sql(sql_script)

In [27]:
output.count()

448

In [28]:
output.show()

+----------------+--------------------+----------+--------------+---------------+--------------------+--------------------+-----------------+--------+----+-----------+--------------+-----+--------------------+
|        category|           review_ID|product_ID|   reviewer_ID|rating_out_of_5|      review_summary|         review_text|review_word_count|verified|vote| reviewTime|unixReviewTime|image|          source_url|
+----------------+--------------------+----------+--------------+---------------+--------------------+--------------------+-----------------+--------+----+-----------+--------------+-----+--------------------+
|AMAZON_FASHION_5|B001IKJOLW-A29K85...|B001IKJOLW|A29K85C92Z8HGR|            4.0|         Light shoes|It's been a long ...|               86|    true|null| 07 7, 2017|    1499385600| null|http://deepyeti.u...|
|AMAZON_FASHION_5|B001IKJOLW-A2JOUO...|B001IKJOLW|A2JOUOASELY1XR|            5.0|I feel like this ...|I feel like this ...|               21|    true|null|06 19

In [ ]:
df = spark.read.json("file://"+SparkFiles.get(file),schema)

In [ ]:
schema = StructType([ 
    StructField("overall",IntegerType(),True), 
    StructField("vote",IntegerType(),True), 
    StructField("verified",BooleanType(),True)
  ])

In [ ]:
df.show()